# 🚀 ChatRoutes Python SDK - Quick Start Guide

[![PyPI](https://img.shields.io/pypi/v/chatroutes.svg)](https://pypi.org/project/chatroutes/)
[![Python](https://img.shields.io/pypi/pyversions/chatroutes.svg)](https://pypi.org/project/chatroutes/)

Welcome to ChatRoutes! This notebook will help you get started with the ChatRoutes Python SDK.

## ⚠️ Beta Release Notice

ChatRoutes is currently in **beta**. The API may change without maintaining backward compatibility. Please use with caution in production environments.

## 🔑 Getting Your API Key

**IMPORTANT:** Before you begin, you need to get your API key:

1. **Visit** [chatroutes.com](https://chatroutes.com)
2. **Sign up** for a free account
3. **Go to Dashboard** → Navigate to the **API section**
4. **Generate** your API key
5. **Copy** the API key and paste it below when prompted

## 🤖 Supported Models

**OpenAI:**
- **`gpt-5`** (default) - OpenAI's GPT-5

**Anthropic Claude 4:**
- **`claude-opus-4-1`** - Claude Opus 4.1 (most capable)
- **`claude-opus-4`** - Claude Opus 4
- **`claude-opus-4-0`** - Claude Opus 4.0
- **`claude-sonnet-4-5`** - Claude Sonnet 4.5 (best for coding)
- **`claude-sonnet-4-0`** - Claude Sonnet 4.0

**Anthropic Claude 3:**
- **`claude-3-7-sonnet-latest`** - Claude 3.7 Sonnet (latest)
- **`claude-3-5-haiku-latest`** - Claude 3.5 Haiku (fastest)

**Important**: Use these exact model names. Other model names are not supported.

---

## 📦 Step 1: Install ChatRoutes SDK

In [ ]:
!pip install -q chatroutes
!pip show chatroutes
print("✅ ChatRoutes SDK installed successfully!")

## 🔐 Step 2: Set Up Your API Key

Enter your API key below. You can get it from [chatroutes.com](https://chatroutes.com) → Dashboard → API section.

In [ ]:
from getpass import getpass
from chatroutes import ChatRoutes

# Enter your API key (it will be hidden)
api_key = getpass('Enter your ChatRoutes API key: ')

# Initialize the ChatRoutes client
client = ChatRoutes(api_key=api_key)

print("✅ ChatRoutes client initialized successfully!")

## 💬 Step 3: Create Your First Conversation

Let's create a conversation and send a message!

In [ ]:
# Create a new conversation
conversation = client.conversations.create({
    'title': 'My First ChatRoutes Conversation',
    'model': 'gpt-5'  # Try: claude-opus-4-1, claude-sonnet-4-5, etc.
})

print(f"✅ Created conversation: {conversation['title']}")
print(f"📝 Conversation ID: {conversation['id']}")
print(f"🤖 Model: {conversation['model']}")

## 📤 Step 4: Send a Message

In [ ]:
# Send a message to the conversation
response = client.messages.send(
    conversation['id'],
    {
        'content': 'Hello! Can you explain what ChatRoutes is in one sentence?',
        'model': 'gpt-5'
    }
)

print("\n🤖 AI Response:")
print(response['message']['content'])
print(f"\n📊 Model: {response['model']}")
print(f"🔢 Tokens used: {response['usage']['totalTokens']}")

## 🌊 Step 5: Try Streaming Responses

ChatRoutes supports **real-time streaming** for faster user experience!

### What is Streaming?

Instead of waiting for the complete response, streaming sends the AI's response **as it's being generated**:

**Non-Streaming (Traditional):**
```
[Wait 3 seconds...] → Full response appears at once
```

**Streaming (Real-Time):**
```
[0.5s] "Why did..."
[1.0s] "Why did the programmer..."
[1.5s] "Why did the programmer quit? Because..."
[2.0s] "Why did the programmer quit? Because they didn't get arrays!"
```

### 🎥 Watch for the Effect!

In the next cell, you'll see the response appear **word-by-word** in real-time. This is how ChatGPT and Claude feel fast and responsive!

In [ ]:
import sys
import time

print("🌊 Streaming response (watch it appear in real-time):\n")
print("─" * 60)

# Track streaming metadata
chunks_received = 0
start_time = time.time()

# Define callback for streaming chunks
def on_chunk(chunk):
    global chunks_received
    if chunk.get('type') == 'content' and chunk.get('content'):
        chunks_received += 1
        # Print chunk immediately with flush
        print(chunk['content'], end='', flush=True)
        sys.stdout.flush()  # Force output in Jupyter

def on_complete(message):
    elapsed = time.time() - start_time
    print(f"\n")
    print("─" * 60)
    print(f"✅ Streaming complete!")
    print(f"📊 Streaming Stats:")
    print(f"   • Chunks received: {chunks_received}")
    print(f"   • Time elapsed: {elapsed:.2f} seconds")
    print(f"   • Message ID: {message['id']}")
    print(f"   • Average chunk rate: {chunks_received/elapsed:.1f} chunks/sec")

print("⏳ Waiting for first chunk...\n")

# Send a message with streaming
client.messages.stream(
    conversation['id'],
    {
        'content': 'Tell me a short joke about programming.',
        'model': 'gpt-5'
    },
    on_chunk=on_chunk,
    on_complete=on_complete
)

print("\n💡 Notice: The response appeared word-by-word, not all at once!")
print("   This is real-time streaming from the AI model.")

## 🌳 Step 6: Create a Branch (Alternative Response)

One of ChatRoutes' unique features is conversation branching - explore alternative responses!

In [ ]:
# Create a branch for alternative responses
branch = client.branches.create(
    conversation['id'],
    {
        'title': 'Alternative Response',
        'contextMode': 'FULL'
    }
)

print(f"✅ Created branch: {branch['title']}")
print(f"🌿 Branch ID: {branch['id']}")

# Send a message to the branch using the branches resource
# NOTE: This creates TWO messages in the branch:
#   1. Your user message (the question)
#   2. The AI's assistant message (the response)
branch_response = client.branches.send_message(
    conversation['id'],
    branch['id'],
    {
        'content': 'Now tell me a joke about AI instead.',
        'model': 'gpt-5'
    }
)

print("\n🌿 Branch response:")
print(branch_response['assistantMessage']['content'])
print("\n💡 Note: Each message exchange creates 2 messages (user + assistant)")

## 🎭 Step 7: Try Different Claude Models

Test various Claude models for different use cases!

In [ ]:
# Claude Opus 4.1 - Most capable
opus_conv = client.conversations.create({
    'title': 'Claude Opus 4.1 - Most Capable',
    'model': 'claude-opus-4-1'
})

opus_response = client.messages.send(
    opus_conv['id'],
    {'content': 'Explain quantum entanglement in simple terms.', 'model': 'claude-opus-4-1'}
)
print("🎭 Claude Opus 4.1 (Most Capable):")
print(opus_response['message']['content'][:200] + "...\n")

# Claude Sonnet 4.5 - Best for coding
sonnet_conv = client.conversations.create({
    'title': 'Claude Sonnet 4.5 - Best for Coding',
    'model': 'claude-sonnet-4-5'
})

sonnet_response = client.messages.send(
    sonnet_conv['id'],
    {'content': 'Write a Python function to check if a number is prime.', 'model': 'claude-sonnet-4-5'}
)
print("💻 Claude Sonnet 4.5 (Best for Coding):")
print(sonnet_response['message']['content'][:200] + "...\n")

# Claude 3.5 Haiku - Fastest
haiku_conv = client.conversations.create({
    'title': 'Claude 3.5 Haiku - Fastest',
    'model': 'claude-3-5-haiku-latest'
})

haiku_response = client.messages.send(
    haiku_conv['id'],
    {'content': 'Summarize AI in one sentence.', 'model': 'claude-3-5-haiku-latest'}
)
print("⚡ Claude 3.5 Haiku (Fastest):")
print(haiku_response['message']['content'])

## 📋 Step 8: List Your Conversations

In [ ]:
# List all conversations
result = client.conversations.list({
    'page': 1,
    'limit': 5,
    'filter': 'all'
})

print(f"📊 Total conversations: {result['total']}")
print(f"📄 Page {result['page']} of {result.get('pages', '?')}")
print(f"➡️ Has next page: {result.get('hasNext', False)}\n")

for conv in result['data']:
    print(f"💬 {conv['title']}")
    print(f"   ID: {conv['id']}")
    print(f"   Messages: {conv.get('messageCount', 0)}")
    print(f"   Branches: {conv.get('branchCount', 0)}")
    print(f"   Created: {conv['createdAt']}")
    print(f"   Updated: {conv['updatedAt']}")
    print()

## 🌲 Step 9: View Conversation Tree

See the structure of your conversation with all its branches!

In [ ]:
# Get conversation tree
tree = client.conversations.get_tree(conversation['id'])

def print_tree_branch(branch_node, indent=0):
    """Print a branch and its messages"""
    prefix = "  " * indent
    branch = branch_node['branch']
    
    print(f"{prefix}🌿 Branch: {branch['title']}")
    print(f"{prefix}   Messages: {branch['_count']['messages']}")
    print(f"{prefix}   Active: {branch['isActive']}")
    
    # Print child branches recursively
    for child in branch_node.get('children', []):
        print_tree_branch(child, indent + 1)

print("🌲 Conversation Tree:\n")
print(f"Conversation: {tree['conversation']['title']}")
print(f"Model: {tree['conversation']['model']}")
print(f"Total branches: {len(tree['branches'])}\n")

# Print all branches
for branch_node in tree['branches']:
    print_tree_branch(branch_node)

print("\n💡 Message Count Explanation:")
print("   Each conversation exchange creates 2 messages:")
print("   • 1 user message (your question)")
print("   • 1 assistant message (AI's response)")
print("   So if you see '4 messages', that's 2 exchanges (2 Q&A pairs)")

## 🧹 Optional: Clean Up

Delete the test conversations if you want to clean up.

In [ ]:
# Uncomment to delete the conversations
# client.conversations.delete(conversation['id'])
# client.conversations.delete(opus_conv['id'])
# client.conversations.delete(sonnet_conv['id'])
# client.conversations.delete(haiku_conv['id'])
# print("✅ Conversations deleted")

print("ℹ️ Uncomment the code above to delete the test conversations")

## 📚 Next Steps

Congratulations! You've learned the basics of ChatRoutes. Here are some resources to help you go further:

### 📖 Documentation
- [GitHub Repository](https://github.com/chatroutes/chatroutes-python-sdk)
- [API Documentation](https://docs.chatroutes.com)
- [Quick Start Guide](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/QUICKSTART.md)

### 💡 Examples
- [Basic Usage](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/basic_usage.py)
- [Streaming](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/streaming_example.py)
- [Branching](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/branching_example.py)
- [Error Handling](https://github.com/chatroutes/chatroutes-python-sdk/blob/main/examples/error_handling.py)

### 🤖 Supported Models

**OpenAI:**
- `gpt-5` (default)

**Anthropic Claude 4:**
- `claude-opus-4-1` (most capable)
- `claude-opus-4`
- `claude-opus-4-0`
- `claude-sonnet-4-5` (best for coding)
- `claude-sonnet-4-0`

**Anthropic Claude 3:**
- `claude-3-7-sonnet-latest`
- `claude-3-5-haiku-latest` (fastest)

### 🔧 Key Features to Explore

1. **Multiple Models**: Switch between 8+ AI models for different use cases
2. **Conversation Branching**: Explore alternative conversation paths
3. **Streaming**: Real-time response streaming
4. **Message History**: Access and manage conversation history
5. **Branch Merging**: Merge branches back into main conversation

### 💬 Support
- Email: support@chatroutes.com
- GitHub Issues: [Report a bug](https://github.com/chatroutes/chatroutes-python-sdk/issues)

---

**Happy coding with ChatRoutes!** 🚀